# 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import re

from tqdm import tqdm

from pymongo import MongoClient
from pymongo.cursor import CursorType

import warnings
warnings.filterwarnings('ignore')

from IPython.display import Image

from pyecharts import Pie

import itertools
import PIL.Image as pilimg

import plotly.express as px
from pyecharts import Pie

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "C:\Users\Bit\anaconda3\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [2]:
from hanspell import spell_checker
from pykospacing import spacing
from soynlp.normalizer import *
from konlpy.tag import Okt

okt = Okt()

In [3]:
stopwords = []
f = open('data/stopwords.csv')
lines = f.readlines()
for line in lines:
    line = line.strip()
    stopwords.append(line)
f.close()

In [4]:
def delete_re_dm(dataframe,column):
    dataframe[column] = dataframe[column].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', " ")
    dataframe1 = dataframe.drop_duplicates(subset=[column], inplace=False)
    dataframe1[column].replace('', np.nan, inplace=True)
    dataframe2 = dataframe1.dropna(how='any')
    dataframe2.reset_index(inplace=True)
    dataframe2.drop(columns = 'index', inplace=True)
    return dataframe2

def delete_dp_dm(dataframe,column):
    dataframe1 = dataframe.copy()
    dataframe1[column] = dataframe1[column].apply(lambda x : x.strip())
    dataframe1[dataframe1[column]==''] = np.nan
    dataframe2 = dataframe1.dropna(how='any')
    dataframe2.reset_index(inplace=True)
    dataframe2.drop(columns = 'index', inplace=True)
    return dataframe2

# 전처리 함수
def text_preprocessing(textlist):
    X_train = []
    i=0
    for sentence in tqdm(textlist):
        try:
            temp_X = []
            new_sentence = spacing(sentence)
            new_sentence = spell_checker.check(new_sentence).checked
            new_sentence = emoticon_normalize(new_sentence, num_repeats=2)
            temp_X = okt.morphs(new_sentence, norm=True, stem=True)
            temp_X = [w for w in temp_X if not w in stopwords]
#             temp_X = [w for w in temp_X if len(w)>=2]
            X_train.append(temp_X)
        except Exception as e:
            temp_X = ['에러']
            X_train.append(temp_X)
            print(i,'번째에서 error 발생', e)
        i+=1
    return X_train

def sentence_preprocessing(sentence):
    new_sentence = spacing(sentence)
    new_sentence = spell_checker.check(new_sentence).checked
    new_sentence = emoticon_normalize(new_sentence, num_repeats=2)
    new_sentence = okt.morphs(new_sentence, norm=True, stem=True)
    new_sentence = [w for w in new_sentence if not w in stopwords]
    return new_sentence

In [5]:
def find_item_one(mongo, condition=None, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].find_one(condition, {"_id": False})
    return result

def find_item(mongo, condition=None, db_name=None, collection_name=None):
    result = mongo[db_name][collection_name].find(condition, {"_id": False}, no_cursor_timeout=True)
    return result

In [6]:
host = ''
port = ''
mongo = MongoClient(host, int(port))
print(mongo)

MongoClient(host=['121.138.83.13:1234'], document_class=dict, tz_aware=False, connect=True)


In [7]:
cursor_happy = find_item(mongo, None,'Twitter','happy')
cursor_angry = find_item(mongo, None,'Twitter','angry')
cursor_sad = find_item(mongo, None,'Twitter','sad')
cursor_surprise = find_item(mongo, None,'Twitter','suprise')
cursor_fear = find_item(mongo, None,'Twitter','fear')
cursor_boring = find_item(mongo, None,'Twitter','boring')


In [8]:
text_happy = []
text_angry = []
text_sad = []
text_fear = []
text_surprise = []
text_boring = []

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
for i in tqdm(cursor_happy):
    text_happy.append(i['text'])

for i in tqdm(cursor_angry):
    text_angry.append(i['text'])
    
for i in tqdm(cursor_sad):
    text_sad.append(i['text'])    
    
for i in tqdm(cursor_fear):
    text_fear.append(i['text'])

for i in tqdm(cursor_surprise):
    text_surprise.append(i['text'])   
    
for i in tqdm(cursor_boring):
    text_boring.append(i['text'])   

104330it [00:02, 41536.08it/s]
250158it [00:06, 41328.36it/s]
98618it [00:02, 42987.20it/s]
133144it [00:03, 36093.15it/s]
127763it [00:03, 38254.88it/s]
157591it [00:02, 58576.23it/s]


In [9]:
df_happy1 = pd.DataFrame({'text':text_happy[:100000]})
df_angry1 = pd.DataFrame({'text':text_angry[:100000]})
df_sad1 = pd.DataFrame({'text':text_sad[:100000]})
df_fear1 = pd.DataFrame({'text':text_fear[:100000]})
df_surprise1 = pd.DataFrame({'text':text_surprise[:100000]})
df_boring1 = pd.DataFrame({'text':text_boring[:100000]})

In [10]:
df_happy = delete_re_dm(df_happy1,'text')
df_happy = delete_dp_dm(df_happy,'text')

df_angry = delete_re_dm(df_angry1,'text')
df_angry = delete_dp_dm(df_angry,'text')

df_sad = delete_re_dm(df_sad1,'text')
df_sad = delete_dp_dm(df_sad,'text')

df_fear = delete_re_dm(df_fear1,'text')
df_fear = delete_dp_dm(df_fear,'text')

df_surprise = delete_re_dm(df_surprise1,'text')
df_surprise = delete_dp_dm(df_surprise,'text')

df_boring = delete_re_dm(df_boring1,'text')
df_boring = delete_dp_dm(df_boring,'text')

In [11]:
df_happy = df_happy.drop(df_happy.index[[1,8,17,26,31,36,39,59,84,98,107,144,165,196,207,272,279,281]])

In [12]:
df_happy = df_happy[df_happy['text'].str.contains('아스트로')==False]
df_happy = df_happy[df_happy['text'].str.contains('은우')==False]
df_happy = df_happy[df_happy['text'].str.contains('트레저')==False]
df_happy = df_happy[df_happy['text'].str.contains('건희')==False]
df_happy = df_happy[df_happy['text'].str.contains('섹트')==False]
df_happy = df_happy[df_happy['text'].str.contains('도운')==False]

In [13]:
df_surprise = df_surprise[df_surprise['text'].str.contains('선착') == False]
df_surprise = df_surprise[df_surprise['text'].str.contains('방탄') == False]
df_surprise = df_surprise[df_surprise['text'].str.contains('원호') == False]
df_surprise = df_surprise[df_surprise['text'].str.contains('영웅') == False]
df_surprise = df_surprise[df_surprise['text'].str.contains('찬열') == False]
df_surprise = df_surprise[df_surprise['text'].str.contains('티저') == False]

In [14]:
df_surprise = df_surprise.drop(df_surprise.index[[124,141,174,194,199,217,233,252,468,543]])

In [15]:
df_fear = df_fear[df_fear['text'].str.contains('바디') == False]
df_fear = df_fear[df_fear['text'].str.contains('변녀') == False]
df_fear = df_fear[df_fear['text'].str.contains('엑소') == False]

In [16]:
df_happy['category'] = 0
df_angry['category'] = 1
df_sad['category'] = 2
df_fear['category'] = 3
df_surprise['category'] = 4
df_boring['category'] = 5

In [17]:
df = pd.concat([df_happy, df_angry, df_sad, df_fear, df_surprise, df_boring])

In [18]:
df.shape

(241542, 2)